In [1]:
import requests

# URL for Budweiser Brewing APAC 2022 ESG Report (for demonstration purpose, assuming PDF link is known)
report_url = "https://www.budweiserapac.com/uploadfiles/2022/04/20220412202949055.pdf"
response = requests.get(report_url)
filename = "BudweiserAPAC_2022_ESG.pdf"
with open(filename, "wb") as f:
    f.write(response.content)
print(f"Downloaded report to {filename}, size: {len(response.content)} bytes")

Downloaded report to BudweiserAPAC_2022_ESG.pdf, size: 36914809 bytes


In [2]:
import fitz

doc = fitz.open(filename)
outline = []        # to store outline entries (page, heading text, level)
text_blocks = []    # to store textual content blocks
table_blocks = []   # to store table content as text or data structures

for page_index in range(doc.page_count):
    page = doc.load_page(page_index)
    blocks = page.get_text("blocks")  # list of text blocks: (x0,y0,x1,y1,text,block_no, block_type)
    blocks.sort(key=lambda b: (b[1], b[0]))  # sort by vertical position, then horizontal
    for block in blocks:
        x0, y0, x1, y1, text, block_no, block_type = block
        if text.strip() == "":
            continue
        # Identify headings (e.g., lines that end with newline and are all caps or have larger font size)
        # For simplicity, let's say if text is short and ends with "\n" it's a heading.
        if len(text) < 50 and text.strip().endswith("\n"):
            level = 1
            # determine level by text formatting (not available via this interface, so placeholder)
            outline.append({"page": page_index+1, "level": level, "title": text.strip()})
        # Identify tables by presence of multiple columns or numeric data (simplified)
        elif any(sep in text for sep in ["|", "\t"]) or text.count("%")>2:
            table_blocks.append({"page": page_index+1, "content": text})
        else:
            text_blocks.append({"page": page_index+1, "content": text})


In [ ]:
from huggingface_hub import login
your_hf_token = "hf-xxx"
login(token=your_hf_token)

d:\ESGReveal\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import AutoProcessor, AutoModelForTokenClassification

model_id = "Mit1208/layoutlmv3-finetuned-DocLayNet"   # ✅ real repo id

layout_processor = AutoProcessor.from_pretrained(model_id)
layout_model = AutoModelForTokenClassification.from_pretrained(model_id)

image = page.get_pixmap().pil()   # your existing PyMuPDF code
inputs = layout_processor(images=image, return_tensors="pt")

outputs = layout_model(**inputs)
# outputs.logits has token-level labels like "text", "title", "table", etc.


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Error while downloading from https://huggingface.co/Mit1208/layoutlmv3-finetuned-DocLayNet/resolve/main/pytorch_model.bin: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.
Trying to resume download...


AttributeError: 'Pixmap' object has no attribute 'pil'

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
